# Intro

In this notebook I experiment with an alternative to the traditional SIR model, with the assumption of homogeneous mixing replaced with a matrix of social connections and with $R_0$ endogenized.

One goal of this model, aside from perhaps getting more reasonable predictions, is to be able to estimate "deeper" parameters (via indirect inference) from social interaction data, empirical estimates of $R_0$, and estimates of the share of infections from super-spreaders. One such parameter, outlined in more detail below, controls whether prolonged exposure is needed to transmit the virus. Then we could use those parameters to assess whether social distancing measures that limit fleeting interactions are useful or not. These include park closures, mass transit cuts, and restrictions on where people can walk as is done now in Italy ([NYT](https://www.nytimes.com/aponline/2020/03/22/business/bc-eu-virus-outbreak-italy-restrictions.html)). As we will probably live with this nightmare for a long time, it will be useful to study which restrictions are actually slowing the disease and which are just driving us insane, and (slowly! carefully!) relax the latter as evidence builds for their inefficacy.

The estimation procedure would be as follows:
- Fill in a symmetrical sparse matrix of social interactions (maybe parameterized based on on smartphone location data). Element $i,j$ tells us how much time $i$ and $j$ spend in close proximity with each other on average, normalized between 0 and 1
- For each row $i$ of the data, simulate forward as if only $i$ was infected. Use a simple parametric function that takes as an input the amount of time $i$ spends with $j$, and spits out a probability that $j$ will get infected. In each period $i$ recovers with some probability and stops spreading new infections
- After simulating forward for a few periods, we get a vector where each element $i$ indicates the number of people that $i$ has infected. The mean of the elements targets $R_0$; the share infections generated by super-spreaders helps pin down a parameter that governs whether sustained interactions are necessary to spread the disease or if even fleeting interactions can do the trick

Please note that I am not an epidemiologist! I was literally just inspired by these twitter threads and posts, and wanted to give it a go. Plus I'm not a terribly advanced coder, and the code below seems far more sophisticated than what I'm doing (although the focus is quite different)


https://twitter.com/Noahpinion/status/1245092626463768576

https://twitter.com/gro_tsen/status/1241745979663155203

https://github.com/Gro-Tsen/epidemiology/blob/master/run-epidemic.pl

https://necsi.edu/review-of-ferguson-et-al-impact-of-non-pharmaceutical-interventions?fbclid=IwAR1E-cL4gnXVE15BnpX5anbt99EbjksKYmTUfjFTe0DdyR23C_YvfbhZ5RY

# The model

For a society of $N$ people, each element $i,j$ of the symmetrical $NxN$ social-connection matrix represents the degree of connectedness between individuals $i$ and $j$. I think of this literally as time spent in close proximity with each other. Susceptibility, infectiousness, and recovery are represented by $Nx1$ vectors populated with zeros and ones. Denote the matrix and vectors as $M,S,I,$ and $R$.

Denote the stochastic recovery function as $\rho$, and note that the $.$ notation below indicates an elementwise operation:

$$\rho(0)=0$$

$$\rho(1)=1 \text{  w.p. } r$$



$$R_{t+1}=R_t +\rho I_t$$

$$S_{t+1}=\vec{1} -R_{t+1}$$

In [1]:
using LinearAlgebra
using SparseArrays
using StatsBase
using Random

In [2]:
function ρ(x; r=0.5)
    y=rand()
    if y<r && x==1
        return 1 # set to 1 if already recovered or if stochastic draw less than recovery prob
    else
        return 0
    end
end
ρ(0)

0

To simulate the spread of the disease: in each period, we calculate $X=M.*I'$, the $NxN$ degree of exposure for each individual from each person in their network.  Each element of this matrix is then fed into the function $\iota$, which takes on value 1 with some probability dependending on the degree of exposure.

Then, if any element $i$ of $\iota.(X[i,:])$ is 1, the $i$th element of vector $\tilde I$ becomes 1. This vector is multiplied elementwise by susceptibility  vector $S$


$$\tilde I[i]=\mathbb{1}[\max(\iota.(X[i,:]))=1]$$


$$I_{t+1}=I_t-\rho.(I_t)+\tilde I .*S_t$$

I parametrize $\iota$ in the following way, to capture the intuition that the duration of each interaction might or might not affect the probability of spreading infection. The parameter $k$ controls whether sustained exposures are necessary. A very low value (~0) would imply that any interaction could spread the disease; a high value indicates that only prolonged exposure would do the trick. And the parameter $e$ is an efficiency term which may be policy-amenable, and which may indeed be estimable from before-and-after studies of mask-wearing policies.

$$\iota(x)=1 \text{ w.p   } (e*x)^k$$

In [3]:
function ι(x; k=3.5, e=1)
    y=rand()
    if y>(e*x)^k # don't switch unless the random number is larger than the degree of social connection
        return 0
    else
        return 1
    end
end

ι (generic function with 1 method)

In [4]:
function Ĩ(M)
    Ĩ=zeros(length(M[1,:]))
    for i in 1:length(Ĩ)
        if maximum(ι.(M[i,:]))==1
            Ĩ[i]=1
        end
    end
    return Ĩ
end

Ĩ (generic function with 1 method)

I initialize $M$ as a random matrix with degrees of connectivity ranging from one to zero. As most connections between most people are very weak I set things up so that the numbers are very small by drawing uniform randoms and then scaling them up by an exponent.

Parameters:

- Sparsity $s$: 0.1
- Degree of connectedness $c$: 10

This is a crude parametrization of the network structure just to get something off the ground. Again, actual data about the number and duration of social contacts (smartphones) can be used to help fill in this matrix in a reasonable way.

# Setting up a simulation

In [5]:
M=sprand(1000,1000,0.1).^10
M=Symmetric(M)
diag(M.data).=0
M.data

1000×1000 SparseMatrixCSC{Float64,Int64} with 100199 stored entries:
  [16 ,    1]  =  0.0398939
  [24 ,    1]  =  0.0094577
  [51 ,    1]  =  0.0129961
  [71 ,    1]  =  0.00564231
  [97 ,    1]  =  5.45874e-7
  [118,    1]  =  0.108733
  [120,    1]  =  0.317333
  [122,    1]  =  8.94724e-7
  [125,    1]  =  0.00109651
  [130,    1]  =  2.196e-6
  [139,    1]  =  0.000678
  [144,    1]  =  0.0494426
  ⋮
  [926, 1000]  =  2.19231e-5
  [929, 1000]  =  0.00112137
  [940, 1000]  =  0.318281
  [941, 1000]  =  0.0125349
  [949, 1000]  =  0.509506
  [955, 1000]  =  5.65372e-10
  [960, 1000]  =  1.41333e-7
  [976, 1000]  =  0.00259902
  [979, 1000]  =  3.92106e-7
  [982, 1000]  =  0.00053864
  [984, 1000]  =  2.37666e-8
  [992, 1000]  =  3.37775e-6
  [999, 1000]  =  0.0604279

This code will be a useful example later:

In [6]:
M2=sparse(deepcopy(M))
M2.nzval.=0
M2

1000×1000 SparseMatrixCSC{Float64,Int64} with 99974 stored entries:
  [15 ,    1]  =  0.0
  [21 ,    1]  =  0.0
  [24 ,    1]  =  0.0
  [36 ,    1]  =  0.0
  [38 ,    1]  =  0.0
  [43 ,    1]  =  0.0
  [47 ,    1]  =  0.0
  [76 ,    1]  =  0.0
  [83 ,    1]  =  0.0
  [86 ,    1]  =  0.0
  [89 ,    1]  =  0.0
  [90 ,    1]  =  0.0
  ⋮
  [926, 1000]  =  0.0
  [929, 1000]  =  0.0
  [940, 1000]  =  0.0
  [941, 1000]  =  0.0
  [949, 1000]  =  0.0
  [955, 1000]  =  0.0
  [960, 1000]  =  0.0
  [976, 1000]  =  0.0
  [979, 1000]  =  0.0
  [982, 1000]  =  0.0
  [984, 1000]  =  0.0
  [992, 1000]  =  0.0
  [999, 1000]  =  0.0

Next I set up the initial vectors, with 1 percent of the population infected and no one recovered

In [7]:
I=zeros(1000,100)
for i in 1:length(I[:,1])
    y=rand()
    if y>0.01
        I[i,1]=0
    else
        I[i,1]=1
    end
end
I
S=ones(1000,100)
S[:,1].=S[:,1].-I[:,1]
R=zeros(1000,100)

1000×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

Next I perform the simulation:

In [8]:

for t in 2:30
    new_recovery=ρ.(I[:,t-1])
    R[:,t].=new_recovery.+R[:,t-1]
    I[:,t].=I[:,t-1].-new_recovery
    S[:,t].=ones(1000).-R[:,t].-I[:,t]
    I[:,t].+=Ĩ(M.*I[:,t]').*S[:,t]
    println("I=",sum(I[:,t]),", S=",sum(S[:,t]),", R=",sum(R[:,t]) )
end

I=15.0, S=992.0, R=3.0
I=23.0, S=982.0, R=11.0
I=29.0, S=966.0, R=23.0
I=41.0, S=948.0, R=39.0
I=53.0, S=920.0, R=62.0
I=100.0, S=885.0, R=83.0
I=117.0, S=817.0, R=136.0
I=142.0, S=747.0, R=195.0
I=160.0, S=663.0, R=265.0
I=148.0, S=575.0, R=362.0
I=166.0, S=490.0, R=422.0
I=144.0, S=412.0, R=499.0
I=117.0, S=357.0, R=567.0
I=99.0, S=316.0, R=624.0
I=78.0, S=277.0, R=674.0
I=59.0, S=248.0, R=711.0
I=34.0, S=230.0, R=743.0
I=16.0, S=223.0, R=764.0
I=10.0, S=220.0, R=773.0
I=3.0, S=217.0, R=780.0
I=1.0, S=217.0, R=782.0
I=1.0, S=217.0, R=782.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0
I=0.0, S=217.0, R=783.0


# Setting up an estimation exercise

- Each row of $\rho.M$ indicates how many people each individual would infect in a week if they were the only person infected.
- We can simulate each row forward--i.e. acting as if that person was the only infected individual and assuming that they did not know they were infected. How many of their connections would get sick?
- The mean of each row after a few weeks of infection can be used to target estimates of R_0.
- The *variance* of each row can be used to target the share of infections from "super spreader" events.
- The $r$ parameter targets recovery times
- Parameters $s$ and $c$ can likely be estimated from smartphone location data--in practice such data could be used to discipline a more realistic parametrization of the network structure
- The parameter $k$ tells us what kinds of social gatherings are more or less likely to spread diseases-close interactions or distant ones. Crucially, this parameter is plausibly policy-invariant (whereas $e$ may be amenable to mask-wearing.)

Now for an example of the simulation/estimation routine:

In [9]:
function switch_to_one(x) # only switch connections to 1 if they weren't switched already
    if x>=1.0
        x=1.0
    end
end

switch_to_one (generic function with 1 method)

In [10]:
# initialize matrix as before
M=sprand(1000,1000,0.1).^10
M=Symmetric(M)
diag(M.data).=0


# M2 has same indices as M but data is switched to zero
M2=deepcopy(M)
M2.data.nzval.=0


I_Sim=zeros(1000) # stores number of infected
R_Sim=ones(1000) # stores how many have recovered
t=0


while t<30
  
    Mb=ι.(M.*R_Sim) # everyone infects their social network unless they have recovered
    M2+=Mb # start filling in matrix of zero infected people
    M2=sparse(M2)
    M2.nzval.=switch_to_one.(M2.nzval)
    #M2.=switch_to_one.(M2) # people can only get infected once
    t+=1
    R_Sim=ρ.(R_Sim) # some share recover
end
M2

1000×1000 SparseMatrixCSC{Float64,Int64} with 3859 stored entries:
  [53 ,    1]  =  1.0
  [197,    1]  =  1.0
  [275,    1]  =  1.0
  [520,    1]  =  1.0
  [702,    2]  =  1.0
  [766,    2]  =  1.0
  [773,    2]  =  1.0
  [776,    2]  =  1.0
  [45 ,    3]  =  1.0
  [78 ,    3]  =  1.0
  [272,    3]  =  1.0
  [529,    3]  =  1.0
  ⋮
  [691,  997]  =  1.0
  [877,  997]  =  1.0
  [950,  997]  =  1.0
  [144,  998]  =  1.0
  [692,  998]  =  1.0
  [949,  998]  =  1.0
  [39 ,  999]  =  1.0
  [81 ,  999]  =  1.0
  [249,  999]  =  1.0
  [477,  999]  =  1.0
  [932,  999]  =  1.0
  [124, 1000]  =  1.0
  [547, 1000]  =  1.0

The mean gives us $R_0$:

In [11]:
sum_infected=M2*ones(1000)
mean(sum_infected)

3.859

The skewness may be used as a metric for how much super-spreaders matter (need actual data on the skewness of new infections generated at the individual level as a moment target)

In [12]:
skewness(sum_infected)

0.8896686349323949

And just out of curiosity:

In [13]:
maximum(sum_infected)

15.0

A bit more on the estimation idea now that we have seen the toy example:

- If most people have many fleeting social interactions, only a small number of people have sustained contact with multiple other individuals, and a high share of infections are from super-spreaders, then the model will want $k$ to be very high. Recall social interaction is on a bounded interval between 0 and 1, so a high $k$ means that only sustained social interaction will spread the disease.

- With two moment targets, both $k$ and $e$ should be allowed to vary.

- If everyone has a small number sustained social interactions, but a subset of people also have a lot of fleeting social interactions, the model will want that subset to act as super spreaders and so will set $k$ very low to make even fleeting interactions potentially infectious.

- This has implications for whether policies that target fleeting social interactions (i.e. locking down playgrounds and parks, restricting public transit, restricting movement outdoors, etc) have any effect.

# A comparison with a standard SIR

Note that the high $R_0$ contrasts with the fact that 20 percent were not infected in the end. In a traditional $SIR$ model, most of the population eventually gets infected even for moderate $R_0$: by the time the susceptible share has fallen enough to generate herd immunity, the mass of currently-infected people is so large that it still generates a ton of extra infections rather than dying off.

This was part of the point of one of the original twitter threads I linked to: most other pandemics seem to have topped out at ~35 percent of the population. A network with inhomogeneous mixing can yield such a result.

Below is a toy SIR with parameters picked to be similar to the one above, where the difference in the final susceptible share is quite clear.

In [14]:
S=ones(100)*0.99
I=ones(100)*0.01
R=zeros(100)
γ=1/2 # recovery rate
R0=3.7



3.7

We need to map $R_0$ onto a measure of weekly transmission $\beta$ for the simulation:

$$R_0=\beta*\frac{1}{1-\gamma}$$

In [15]:
β=R0*(1-γ)/1

1.85

In [16]:
for t in 2:30
    
    S[t]=max(S[t-1]-β*I[t-1]*S[t-1],0)
    R[t]=min(R[t-1]+γ*I[t-1],1)
    I[t]=min(max(β*I[t-1]*S[t-1]-γ*I[t-1]+I[t-1],0),1)
    println("I=",round(I[t], digits=4),", S=",round(S[t], digits=4),", R=",round(R[t], digits=4) )
end


I=0.0233, S=0.9717, R=0.005
I=0.0536, S=0.9298, R=0.0167
I=0.1189, S=0.8376, R=0.0434
I=0.2438, S=0.6533, R=0.1029
I=0.4165, S=0.3587, R=0.2248
I=0.4846, S=0.0823, R=0.433
I=0.3161, S=0.0085, R=0.6754
I=0.163, S=0.0035, R=0.8334
I=0.0826, S=0.0025, R=0.9149
I=0.0417, S=0.0021, R=0.9562
I=0.021, S=0.0019, R=0.9771
I=0.0106, S=0.0019, R=0.9876
I=0.0053, S=0.0018, R=0.9929
I=0.0027, S=0.0018, R=0.9955
I=0.0013, S=0.0018, R=0.9969
I=0.0007, S=0.0018, R=0.9975
I=0.0003, S=0.0018, R=0.9979
I=0.0002, S=0.0018, R=0.998
I=0.0001, S=0.0018, R=0.9981
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982
I=0.0, S=0.0018, R=0.9982


# Notes on use cases and merits/demerits

Counterfactuals:

- Social distancing can be thought of as shutting down all but the $N$ closest connections between people, and measures like closing down parks or mass transit can be thought of as shutting down all connections below a threshold length of time
- Masks can be thought of as a technology that affects $e$, and as noted earlier the change may be identifiable from before-and-after studies

Advantages:

- Easy procedure for "structural" parameter estimation
- Should be able to generate super-spreader events
- Discrete number of individuals means that, as is supposedly the case in China, the number of infections can eventually be driven down to zero and need not spring up once quarantines are removed
- Models transmission as a network and drops the assumption of homogeneous mixing
- Uses available data to estimate plausibly policy-invariant parameters, which can then inform social distancing measures
- As in the inhomogeneous mixing code in the twitter link from before, this arbitrary parametrization seems to generate a high initial $R_0$ but a low final share of infected individuals relative to a traditional $SIR$ with the same $R_0$, more in line with historical pandemics

Disadvantages:
- For realistic network sizes this would take up a ton of memory, but sparsity should help (graphs?)
- Currently:
    - It only differentiates social interactions by duration, not physical proximity
    - It treats infectiousness as a binary variable, whereas viral load seems to matter a lot (very relevant for policies that limit fleeting interactions). Still, that probably yields upwards-biased estimates of the benefits of those policies, which makes it a conservative method
    - It ignores environmental transmission (which conversely generates bias in the "wrong" direction)
- The counterfactual suggested above ignores that more social distancing measures mean the closest contacts get even closer, i.e. stronger family transmission from lockdowns. Again, it arguably makes the model more conservative in a useful way
